In [ ]:
import keras
keras.__version__

### 1970년 중반 보스턴 외곽 지역의 범죄율, 지방세율 등의 데이터가 주어졌을 때 주택 가격의 중간 값을 예측

In [ ]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
# 데이터 포인트가 506개로 비교적 개수가 적고 404개는 훈련 샘플, 102개는 테스트 샘플로 나뉨
print(train_data.shape)
print(test_data.shape)
# 입력 데이터에 있는 각 특성은 스케일이 서로 다름 (ex 범죄율)
# 어떤 값은 0 과 1 사이의 비율을 나타내고, 어떤 것은 1 과 12 사이의 값을 가지거나 1 과 100 사이의 값을 가짐

# 13개 데이터의 특성
1. Per capita crime rate.
1. Proportion of residential land zoned for lots over 25,000 square feet.
1. Proportion of non-retail business acres per town.
1. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
1. Nitric oxides concentration (parts per 10 million).
1. Average number of rooms per dwelling.
1. Proportion of owner-occupied units built prior to 1940.
1. Weighted distances to five Boston employment centres.
1. Index of accessibility to radial highways.
1. Full-value property-tax rate per $10,000.
1. Pupil-teacher ratio by town.
1. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
1. % lower status of the population.

In [ ]:
train_targets # 타겟은 주택의 중간 가격으로 (단위:1000달러)

In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
# 상이한 스케일을 가진 값은 신경망에 문제가 됨
# 다양한 데이터에 자동으로 맞추려고 할 수 있겠지만 학습을 더 어렵게 만들기 때문에
# 이런 데이터를 다룰 땐 대표적인 방법으로 정규화를 함


# 각 특성에 대해 특성의 평균을 뺀 뒤, 표준 편차로 나눔
# 특성의 중앙이 0 근처에 맞춰지고 표준 편차가 1이 됨
test_data -= mean
test_data /= std

In [ ]:
####### 이때 주목해야 할 점
####### 테스트 데이터를 정규화 할 때 사용한 값이 훈련 데이터에서 계산한 값 임을 주목
####### 머신 러닝 작업 과정에서 절대로 테스트 데이터에서 계산한 어떤 값도 사용해서는 안 됨 // 데이터 정규화처럼 간단한 작업조차도 하면 안됨

In [ ]:
from keras import models
from keras import layers
import tensorflow as tf

def build_model():
  # 동일한 모델을 여러번 생성할 것이므로 함수를 만들어 사용

  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
                         input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  # 이 네트워크의 마지막 층은 하나의 유닛을 가지고 있고 활성화 함수가 없음 (선형 층 이라고 함)
  # 전형적인 스칼라 회귀 (하나의 연석적인 값을 예측하는 회귀) 를 위한 고성
  # "mes = mean_squared_error" 의 약자

  # model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  # KeyError: 'val_mean_absolute_error' 해결법
  # 이렇게 작성 된 것을, 아래처럼 바꾸라는 것
  model.compile(optimizer='rmsprop', loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])
  return model

In [ ]:
# K-겹 검증을 사용한 훈련 검증
# (훈련에 사용할 에포크의 수 같은) 매개변수들을 조정하면서 모델을 평가하기 위해 이전 예제에서 했던 것처럼 데이터를 훈련 세트와 검증 세트로 나눕니다.
# 데이터 포인트가 많지 않기 때문에 검증 세트도 매우 작아집니다(약 100개의 샘플). 결국 검증 세트와 훈련 세트로 어떤 데이터 포인트가 선택됐는지에 따라 검증 점수가 크게 달라집니다.
# 검증 세트의 분할에 대한 검증 점수의 분산이 높습니다. 이렇게 되면 신뢰있는 모델 평가를 신뢰있게 할 수 없습니다.
# 이런 상황에서 가장 좋은 방법은 K-겹 교차 검증을 사용하는 것입니다.
# 데이터를 K개의 분할(즉, 폴드)로 나누고(일반적으로 K = 4 또는 5), K개의 모델을 각각 만들어 K - 1개의 분할에서 훈련하고 나머지 분할에서 평가하는 방법입니다.
# 모델의 검증 점수는 K 개의 검증 점수의 평균이 됩니다.

In [ ]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
  print("처리중인 폴드 #", i)
  # 검증 데이터 준비 : k번째 분할
  val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples : (i + 1) * num_val_samples]

  # 훈련 데이터 준비 : 다른 분할 전체
  partial_train_data = np.concatenate(
      [train_data[ : i * num_val_samples],
       train_data[(i + 1) * num_val_samples : ]],
       axis = 0)
  partial_train_targets = np.concatenate(
      [train_targets[ : i * num_val_samples],
       train_targets[(i + 1) * num_val_samples : ]],
       axis = 0)
  
  # keras 모델 구성 (compile 포함) // 윗윗 칸에서 만듦
  model = build_model()
  
  # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않음
  model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
  
  # 검증 세트로 모델 평가
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

In [ ]:
print(all_scores)

In [ ]:
print(np.mean(all_scores))

In [ ]:
from keras import backend as K

# 메모리 해제
K.clear_session()

In [ ]:
num_epochs = 500
all_mae_histories = []

k = 4
num_val_samples = len(train_data) // k

for i in range(k):
  print("처리중인 폴드 #", i)
  # 검증 데이터 준비
  val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples : (i + 1) * num_val_samples]

  # 훈련 데이터 준비
  partial_train_data = np.concatenate(
      [train_data[ : i * num_val_samples],
       train_data[(i + 1) * num_val_samples : ]],
       axis = 0)
  partial_train_targets = np.concatenate(
      [train_targets[ : i * num_val_samples],
       train_targets[(i + 1) * num_val_samples : ]],
       axis = 0)
  
  # keras 모델 
  # KeyError: 'val_mean_absolute_error' 해결법 // build_model() 안에 주석으로 적어 둠
  model = build_model() 
  
  # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않음
  history = model.fit(partial_train_data, partial_train_targets, validation_data=(val_data, val_targets), epochs=num_epochs, batch_size=1, verbose=0)
  
  # 검증 세트로 모델 평가
  mae_history = history.history['val_mean_absolute_error']
                                #val_mean_absolute_error
  all_mae_histories.append(mae_history)

In [ ]:
# num_epochs = 500
# all_mae_histories = []
# for i in range(k):
#     print('처리중인 폴드 #', i)
#     # 검증 데이터 준비: k번째 분할
#     val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
#     val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

#     # 훈련 데이터 준비: 다른 분할 전체
#     partial_train_data = np.concatenate(
#         [train_data[:i * num_val_samples],
#          train_data[(i + 1) * num_val_samples:]],
#         axis=0)
#     partial_train_targets = np.concatenate(
#         [train_targets[:i * num_val_samples],
#          train_targets[(i + 1) * num_val_samples:]],
#         axis=0)

#     # 케라스 모델 구성(컴파일 포함)
#     model = build_model()
#     # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
#     history = model.fit(partial_train_data, partial_train_targets,
#                         validation_data=(val_data, val_targets),
#                         epochs=num_epochs, batch_size=1, verbose=0)
#     mae_history = history.history['val_mean_absolute_error']
#     all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')

plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')

plt.show()

In [ ]:
# 새롭게 컴파일 된 모델
model = build_model()

# 전체 데이터로 훈련
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score